In [1]:
import numpy as np
import pandas as pd
import urllib.request
import re

from IPython.display import clear_output

from nltk.lm import MLE   # N-Gram Model
from nltk.lm.preprocessing import padded_everygram_pipeline   # Preprocessing for N-Gram.

In [2]:
sum_df = pd.read_csv("/home/x-mbemus/Desktop/cleaner_summaries.csv")   # Import summary document.
sum_df.head()

,Unnamed: 0,id,title,author,pub_date,genres,summary,link,is_novel,sum_len,file_name
0,0,986,The Plague,Albert Camus,1947,novel,The text of The Plague is divided into five p...,https://www.gutenberg.org/files/30062/30062.txt,True,1120,30062_30062.txt
1,2,6921,Carmilla,Sheridan Le Fanu,1872,fiction,The story is presented by Le Fanu as part of ...,https://www.gutenberg.org/files/10007/10007-0.txt,False,934,10007_10007-0.txt
2,3,7923,Dracula,Bram Stoker,1897,horror,"The novel is told in epistolary format, as a ...",https://www.gutenberg.org/files/345/345-0.txt,True,986,345_345-0.txt
3,4,7923,Dracula,Bram Stoker,1897,horror,"The novel is told in epistolary format, as a ...",https://www.gutenberg.org/files/45839/45839-0.txt,True,986,45839_45839-0.txt
4,5,8237,Don Quixote,Miguel de Cervantes,1605,novel,"The First Sally Alonso Quijano, the protagoni...",https://www.gutenberg.org/files/996/996-0.txt,True,1592,996_996-0.txt


In [3]:
start_token = "*** START OF THIS PROJECT GUTENBERG EBOOK"   # Theoretical divider for start of texts.

end_token = "*** END OF THIS PROJECT GUTENBERG EBOOK"   # Theoretical divider for end of texts.

print(sum_df.at[1, "link"])   # Sample entry.

f = open("/home/x-mbemus/Desktop/text_files/" + sum_df.at[1, "file_name"], "r")   # Opening file.

j = 0   # Iterable

for i in f:    # Iterate through file.
    if j == 22:   # Find line 22.
        x = str(i)[2:-5]   # Capture start of line.
        print(x[0:len(start_token)] == start_token)   # Compare to start_token.
    j += 1   # Increase iterable.

# More testing for start tokens.
#for i in f:
#    try:
#        x = re.sub("\\n", "", i.decode("utf-8"))
    
        #if actv == True:
        
        #if x.strip() != "":
#        print(x)
    
        #if x[0:len(start_token)] == start_token:
            #actv == True
#    except:
#        pass

https://www.gutenberg.org/files/10007/10007-0.txt
False


In [4]:
# Attempting to better identify last sentences.

f = open("/home/x-mbemus/Desktop/text_files/" + sum_df.at[0, "file_name"], "r")   # Open first text.

# Theoretical start and end tokens.
start_token = "*** START OF THIS PROJECT GUTENBERG EBOOK"

end_token = "*** END OF THIS PROJECT GUTENBERG EBOOK"

actv = False   # Will be active when copying begins.

y = ""   # Empty string.
for i in f:   # Iterate through text.
    x = str(i)[2:-5]   # Select first line.
    x = re.sub("_+", "", x)   # Remove underscores.
    x = re.sub("\s+", " ", x)   # Remove extra whitespace.
    
    if x[0:len(end_token)] == end_token:   # Check if it's the end token. 
        actv = False   # Stop copying.
    
    if actv == True and x != "":   # If line isn't empty and copying is active.
        line = x.split(" ")   # Break the line into space-separated tokens.
        for j in line:   # For each toekn.
            if re.search("[\.\?!]", j) == None:   # If there's no punctuation.
                y += j + " "   # Add a space after it. 
            else:   # If there is puncuation.
                y += j    # No space.
                if re.search("^[0-9\.]", y) == None:   # If it starts with numbers or a period.
                    print(y.strip())   # Print.
                y = ""    # Start a new line.
                
        
    if x[0:len(start_token)] == start_token:    # If we match start token.
        actv = True   # Start copying.

In [3]:
#test_index = list(sum_df.sample(frac=.2).index)

#train_index = []   # Need to pick all indecies not in test set.
#for i in sum_df.index:   # A for loop is probably not the fastest way of doing this, but it works.
#    if i not in test_index:
#        train_index.append(i)

#train_df = sum_df.loc[train_index][:]   # Create train set.
#test_df = sum_df.loc[test_index][:]   # Create test set.

#train_df.reset_index(inplace = True, drop=True)
#test_df.reset_index(inplace = True, drop=True)

#train_df.to_csv("/home/x-mbemus/Desktop/text_train_data.csv", index=False)
#test_df.to_csv("/home/x-mbemus/Desktop/text_test_data.csv", index=False)

# Reading in files.
train_df = pd.read_csv("/home/x-mbemus/Desktop/text_train_data.csv")
test_df = pd.read_csv("/home/x-mbemus/Desktop/text_test_data.csv")

In [6]:
# https://clementbm.github.io/theory/2021/12/23/rouge-bleu-scores.html
#!pip install sacrebleu
#!pip install rouge

# Importing metrics.
from sacrebleu.metrics import BLEU
bleu_scorer = BLEU(effective_order=True)

from rouge import Rouge
rouge_scorer = Rouge()

In [7]:
# Checking for average summary length.
sum_len = []   # List to contain lengths.
for i in train_df.loc[:]["summary"]:   # For each summary.
    sum_len.append(len(i.split(" ")))   # Count space-separated tokens.
sent_gen = round(np.mean(sum_len))   # take mean of all summaries.
print(sent_gen)   # print mean.

605


In [ ]:
# Start getting results for training.
# Dict to store results.
results = {"title":[], "target":[], "pred":[], "BLEU":[], "ROUGE":[]}

exceptions = []   # List to hold any problems we run into.

for h in train_df.index:   # Begin iterating.
    try:
        actv = False   # Start without copying.

        f = open("/home/x-mbemus/Desktop/text_files/" + sum_df.at[h, "file_name"], "r")   # Open file.
    
        data = []   # Store sentences here.
        y = []    # Store words here.
    
        for i in f:   # Iterate through file.
            # Text preprocessing for every line.
            x = re.sub("_+", "", i)   # Remove underscores.
            x = re.sub("\s+", " ", x)   # Remove extra spaces.
            x = re.sub("\\n", "", x)   # Remove linebreaks.

            if x != "":    # If x isn't empty.
                line = x.split(" ")   # Collect space-separated tokens.
                for j in line:   # For each token.
                    y.append(j)   # Add to list.
                    if re.search("[\.\?!]", j) != None:   # If we find punctuation.
                        data.append(y)   # Add sentence to sentence list.
                        y = []   # Start new sentence.
            
        target = train_df.at[h, "summary"]   # Extract target summary.
        
        for k in [1, 2, 3, 4, 5]:   # Testing each n-gram in given list.
        
            train_data, vocab = padded_everygram_pipeline(k, data)   # Process sentence list.
        
            ngram = MLE(k)   # Assign model.
            ngram.fit(train_data, vocab)   # Fit model.
        
            output = ngram.generate(sent_gen, random_seed=42)   # Generate mean number of characters.
        
            full_output = ""   # String to store output.
    
            for i in output:   # For token in output.
                if re.search("</?s>", i) == None:   # If not a start/stop token.
                    full_output += i + " "   # Add to sentence.
        
            full_output = full_output.strip()   # Remove extra spaces.
        
            # Score on metrics.
            blue = bleu_scorer.sentence_score(hypothesis=full_output, references=[target])
            red = rouge_scorer.get_scores(hyps=full_output, refs=target)
        
            # Add results to dictionary.
            results["title"].append(train_df.at[h, "title"])
            results["target"].append(target)
            results["n"].append(k)
            results["pred"].append(full_output)
            results["BLEU"].append(blue.score)
            results["ROUGE"].append(red[0]["rouge-l"]["f"])
    
    except Exception as e:   # Report any issues.
        exceptions.append((h, e))
    
    # Track process.
    clear_output(wait=True)
    print(f'{h+1}/{len(train_df.index)}')

train_results = pd.DataFrame(results)   # Convert dictionary to data frame.
train_results.to_csv("/home/x-mbemus/Desktop/NGRAM_train_result.csv", index=False)   # Save file.

648/666


In [13]:
len(exceptions)   # Count exceptions. 

275

In [17]:
for i in [1, 2, 3, 4, 5]:   # Check results.
    print(np.mean(train_results.loc[train_results["n"] == i]["BLEU"]))
    print(np.mean(train_results.loc[train_results["n"] == i]["ROUGE"]))

0.4578767076863248
0.1172296911575709
0.6168586452479008
0.11836204904804738
0.08115851441234766
0.02952460613606504
0.08804820281626032
0.02958704111494906
0.06209394881221388
0.029053551100357036


In [19]:
train_df['flagged'] = 0   # If we had any exceptions, flag it.
for i in exceptions:
    train_df.at[i[0], "flagged"] = 1

In [20]:
train_df.head()

,Unnamed: 0,id,title,author,pub_date,genres,summary,link,is_novel,sum_len,file_name,flagged
0,2,6921,Carmilla,Sheridan Le Fanu,1872,fiction,The story is presented by Le Fanu as part of ...,https://www.gutenberg.org/files/10007/10007-0.txt,False,934,10007_10007-0.txt,0
1,3,7923,Dracula,Bram Stoker,1897,horror,"The novel is told in epistolary format, as a ...",https://www.gutenberg.org/files/345/345-0.txt,True,986,345_345-0.txt,0
2,4,7923,Dracula,Bram Stoker,1897,horror,"The novel is told in epistolary format, as a ...",https://www.gutenberg.org/files/45839/45839-0.txt,True,986,45839_45839-0.txt,0
3,5,8237,Don Quixote,Miguel de Cervantes,1605,novel,"The First Sally Alonso Quijano, the protagoni...",https://www.gutenberg.org/files/996/996-0.txt,True,1592,996_996-0.txt,0
4,9,13535,Heart of Darkness,Joseph Conrad,1899,novel,'Heart of Darkness' opens in first person nar...,https://www.gutenberg.org/files/219/219-0.txt,True,3127,219_219-0.txt,0


In [21]:
train_df.to_csv("/home/x-mbemus/Desktop/text_test_data2.csv")   # Save new train flagged data set.

In [6]:
# Now start testing.
# Dictionary to store results.
results = {"title":[], "target":[], "n":[], "pred":[], "BLEU":[], "ROUGE":[]}

exceptions = []   # List of exceptions.

for h in test_df.index:   # Iterate through test observations.
    try:
        actv = False    # Don't copy text.
        
        # Open file.
        f = open("/home/x-mbemus/Desktop/text_files/" + test_df.at[h, "file_name"], "r")
    
        # Store sentences and words.
        data = []
        y = []
        
        for i in f:   # Iterate through words.
            # Preprocess.
            x = re.sub("_+", "", i)
            x = re.sub("\s+", " ", x)
            x = re.sub("\\n", "", x)

            if x != "":   # Split lines into tokens.
                line = x.split(" ")
                for j in line:
                    y.append(j)   # Add token to sentence.
                    if re.search("[\.\?!]", j) != None:   # If punctuation.
                        data.append(y)   # Add sentence to sentence list.
                        y = []    # Start new sentence.
            
        target = test_df.at[h, "summary"]   # Open target.
        
        # Format data.
        train_data, vocab = padded_everygram_pipeline(2, data)
        
        # Create model.
        ngram = MLE(2)
        ngram.fit(train_data, vocab)
        
        # Generate new summary.
        output = ngram.generate(605, random_seed=42)
        
        full_output = ""   # String for storage.
    
        for i in output:   # For each output word
            if re.search("</?s>", i) == None:   # If not start or stop token.
                full_output += i + " "   # Add to list with space.
        
        full_output = full_output.strip()   # Remove extra spaces.
        
        # Compute metrics.
        blue = bleu_scorer.sentence_score(hypothesis=full_output, references=[target])
        red = rouge_scorer.get_scores(hyps=full_output, refs=target)
        
        # Add results to dictionary.
        results["title"].append(test_df.at[h, "title"])
        results["target"].append(target)
        results["pred"].append(full_output)
        results["BLEU"].append(blue.score)
        results["ROUGE"].append(red[0]["rouge-l"]["f"])
    
        # Observation does not need flag.
        test_df.at[h, "flagged"] = 0
        
    except Exception as e:
        exceptions.append((h, e))   # Add exception to exceptions list.
        test_df.at[h, "flagged"] = 1   # Flag observation.
    
    # Output current status.
    clear_output(wait=True)
    print(f'{h+1}/{len(test_df.index)}')

# Convert to dataframe and save.
test_results = pd.DataFrame(results)
ttest_results.to_csv("/home/x-mbemus/Desktop/NGRAM_test_result.csv", index=False)

# Error is caused because I accidentally left "n" in dictionary and forgot to fill it.

167/167


ValueError: All arrays must be of the same length

In [11]:
# Fix mistake causing error above.
result2 = {}   # New result dictionary.
for i in ["title", "target", "pred", "BLEU", "ROUGE"]:   # For desired keys.
    result2[i] = results[i]   # Add those results to new dictionary.
test_results = pd.DataFrame(result2)   # Create new data frame.
test_results.to_csv("/home/x-mbemus/Desktop/NGRAM_test_result.csv", index=False)   # Save dataframe.

In [12]:
# Checking current results.
print(np.mean(test_results.loc[:]["BLEU"]))
print(np.mean(test_results.loc[:]["ROUGE"]))

0.8010884307295432
0.13885795183534103


In [14]:
test_df.to_csv("/home/x-mbemus/Desktop/text_test_dataf.csv", index=False)    # Save flagged test file.

In [7]:
# More flag testing.
for i in train_df.index:
    try:
        f = open("/home/x-mbemus/Desktop/text_files/" + train_df.at[i, "file_name"], "r")
        x = f.readlines()
        train_df.at[i, "flagged"] = 0
    except:
        train_df.at[i, "flagged"] = 1
    
    clear_output(wait=True)    
    print(f'{i+1}/{len(train_df.index)}')

666/666


In [9]:
len(train_df.index[train_df['flagged'] == 1])   # Check number flagged.

255

In [15]:
# Open test result file.
test_out = pd.read_csv("/home/x-mbemus/Desktop/NGRAM_test_result.csv")

# Add some new metrics to measure.
for i in test_out.index:
    test_out.at[i, "targ_len"] = len(test_out.at[i, "target"].split(" "))
    test_out.at[i, "pred_len"] = len(test_out.at[i, "pred"].split(" "))
    test_out.at[i, "len_diff"] = test_out.at[i, "pred_len"] - test_out.at[i, "targ_len"]

# Print average of those.
print(np.mean(test_out.loc[:]['targ_len']))
print(np.mean(test_out.loc[:]['pred_len']))
print(np.mean(test_out.loc[:]['len_diff']))

# "title", "target", "pred", "BLEU", "ROUGE"

738.4
568.6
-169.8
